# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
from azureml.core import Workspace, Experiment

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()
experiment_name = 'hyperdrive'

experiment=Experiment(ws, experiment_name)

In [4]:
from azureml.data.dataset_factory import TabularDatasetFactory

csv_file = 'https://raw.githubusercontent.com/SwapnaKategaru/Project3/main/heart_failure_clinical_records_dataset.csv'

data = TabularDatasetFactory.from_delimited_files(csv_file)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.core.script_run_config import ScriptRunConfig
from azureml.train.hyperdrive import choice
import os

In [6]:
%%writefile conda_dependencies.yml
dependencies:
- python=3.6.2
- scikit-learn=0.21.2
- pip:
  - azureml-defaults 

Overwriting conda_dependencies.yml


In [7]:
if "training" not in os.listdir():
    os.mkdir("./training")
    
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=100, slack_factor=0.2, slack_amount=None, delay_evaluation=200)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
       "--C": uniform(0.0001, 1),
       "--max_iter": choice(80, 100, 120, 140, 180)
   })

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory=".", compute_target='notebook138660', 
                vm_size="STANDARD_DS2_V2", vm_priority='lowpriority', entry_script='train.py')

hyperdrive_run_config = HyperDriveConfig(param_sampling, primary_metric_name='Accuracy', primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=4, max_concurrent_runs=4, policy=early_termination_policy, estimator=estimator)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
#TODO: Submit your experiment
from azureml.widgets import RunDetails
remote_run = experiment.submit(hyperdrive_run_config)
RunDetails(remote_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [10]:
best_run = remote_run.get_best_run_by_primary_metric()
best_run.download_file("outputs/model.joblib", "model.joblib")

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [10]:
'''import joblib
# Get your best run and save the model from that run.
model = remote_run.get_best_run_by_primary_metric(include_failed=True, include_canceled=True, include_resume_from_runs=True)
remote_run.wait_for_completion(show_output=True)

best_run = remote_run.get_metrics()           
joblib.dump(value=best_run, filename="./outputs.joblib")'''


RunId: HD_17ad486c-d982-4196-ab77-b0f63d87035e
Web View: https://ml.azure.com/experiments/hyperdrive/runs/HD_17ad486c-d982-4196-ab77-b0f63d87035e?wsid=/subscriptions/d4ad7261-832d-46b2-b093-22156001df5b/resourcegroups/aml-quickstarts-138600/workspaces/quick-starts-ws-138600

Execution Summary
RunId: HD_17ad486c-d982-4196-ab77-b0f63d87035e
Web View: https://ml.azure.com/experiments/hyperdrive/runs/HD_17ad486c-d982-4196-ab77-b0f63d87035e?wsid=/subscriptions/d4ad7261-832d-46b2-b093-22156001df5b/resourcegroups/aml-quickstarts-138600/workspaces/quick-starts-ws-138600



['./outputs.joblib']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [11]:
# best_run.register_model(model_name = 'best_automl_model.pkl', model_path = './outputs/')
import joblib
joblib.load(filename='outputs/model.joblib')

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

In [26]:
from azureml.core.model import Model
model = Model.register(model_name='hyperdriverun', model_path='outputs/model.joblib', workspace=ws)

Registering model hyperdriverun


In [13]:
import sklearn

from azureml.core.environment import Environment

environment = Environment("LocalDeploy")
environment.python.conda_dependencies.add_pip_package("inference-schema[numpy-support]")
environment.python.conda_dependencies.add_pip_package("joblib")
environment.python.conda_dependencies.add_pip_package("scikit-learn=={}".format(sklearn.__version__))

In [31]:
# pip install joblib

Note: you may need to restart the kernel to use updated packages.


In [37]:
# %%writefile score.py

import json
import numpy as np
import pandas as pd
import os
from sklearn.externals import joblib
def init():
    global model
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'model.joblib')
    print("Found model:", os.path.isfile(model_path)) #To check whether the model is actually present on the location we are looking at
    model = joblib.load(model_path)
def run(data):
    try:
        data = np.array(json.loads(data))
        data = pd.DataFrame.from_dict(data)
        result = model.predict(data)
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

In [38]:
model

Model(workspace=Workspace.create(name='quick-starts-ws-138660', subscription_id='510b94ba-e453-4417-988b-fbdc37b55ca7', resource_group='aml-quickstarts-138660'), name=hyperdriverun, id=hyperdriverun:1, version=1, tags={}, properties={})

In [39]:
Model(ws, 'hyperdriverun')

Model(workspace=Workspace.create(name='quick-starts-ws-138660', subscription_id='510b94ba-e453-4417-988b-fbdc37b55ca7', resource_group='aml-quickstarts-138660'), name=hyperdriverun, id=hyperdriverun:1, version=1, tags={}, properties={})

In [34]:
print(service.get_logs())

None


In [35]:
pip install --upgrade --upgrade-strategy eager azureml-sdk

     |████████████████████████████████| 2.1 MB 6.6 MB/s eta 0:00:01
     |████████████████████████████████| 116 kB 27.8 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 1.9 MB/s  eta 0:00:01
Requirement already up-to-date: azure-mgmt-keyvault<7.0.0,>=0.40.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-core~=1.22.0->azureml-sdk) (2.2.0)
Requirement already up-to-date: ruamel.yaml>=0.15.35 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-core~=1.22.0->azureml-sdk) (0.16.12)
Requirement already up-to-date: azure-mgmt-resource<15.0.0,>=1.2.1 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-core~=1.22.0->azureml-sdk) (12.0.0)
Requirement already up-to-date: pathspec in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-core~=1.22.0->azureml-sdk) (0.8.1)
Requirement already up-to-date: jmespath in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-core~=1.22.0->azur

In [113]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1,  enable_app_insights=True, auth_enabled=True)

from azureml.core.webservice import LocalWebservice, Webservice
inference_config = InferenceConfig(entry_script='score.py',
                                    environment=environment)
# deployment_config = LocalWebservice.deploy_configuration(port=8890)
service = Model.deploy(ws, "test1", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...............................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://c05355da-4e21-43f4-999d-3893163d1a13.southcentralus.azurecontainer.io/score
http://c05355da-4e21-43f4-999d-3893163d1a13.southcentralus.azurecontainer.io/swagger.json


In [114]:
primary, secondary = service.get_keys()
print(primary)

DiDq37HlNN3IuMh6wWUlSgWS4Z1KCSis


In [115]:
import requests
import json

scoring_uri = 'http://c05355da-4e21-43f4-999d-3893163d1a13.southcentralus.azurecontainer.io/score'

key = 'DiDq37HlNN3IuMh6wWUlSgWS4Z1KCSis'


# Sample data to send to the service
data = json.dumps(
    {"data":
            [
                {
                    'age': 75.0,
                    'anaemia': 0,
                    'creatinine_phosphokinase': 582,
                    'diabetes': 0,
                    'ejection_fraction': 20,
                    'high_blood_pressure': 1,
                    'platelets': 265000.0,
                    'serum_creatinine': 1.9,
                    'serum_sodium': 130,
                    'sex': 1,
                    'smoking': 0,
                    'time': 4
                },
                {
                    'age': 55.0,
                    'anaemia': 0,
                    'creatinine_phosphokinase': 7861,
                    'diabetes': 0,
                    'ejection_fraction': 38,
                    'high_blood_pressure': 0,
                    'platelets': 263358.03,
                    'serum_creatinine': 1.1,
                    'serum_sodium': 136,
                    'sex': 1,
                    'smoking': 0,
                    'time': 6
                }
            ] 
        }
)
                

headers = {'Content-Type': 'application/json'}

headers['Authorization'] = f'Bearer {key}'

resp = requests.post(scoring_uri, data, headers=headers)
print(resp.text)


[0, 0]


# _ _ _ _  _   _ _ _ _ __ _ _ _  _   _ _ _ _ __ _ _ _  _   _ _ _ _ __ _ _ _  _   _ _ _ _ __ _ _ _  _   _ _ _ _ __ _ _ _  _   _ _ _ _ _

In [104]:
from azureml.core.webservice import LocalWebservice
from azureml.core.model import InferenceConfig
inference_config = InferenceConfig(entry_script='score.py', environment=environment) 

# This is optional, if not provided Docker will choose a random unused port.
deployment_config = LocalWebservice.deploy_configuration(port=1234)

local_service = Model.deploy(ws, "deploy", [model], inference_config, deployment_config)

local_service.wait_for_deployment()

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry a99fa470afc343d8a4731cc11e3d7142.azurecr.io
Logging into Docker registry a99fa470afc343d8a4731cc11e3d7142.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM a99fa470afc343d8a4731cc11e3d7142.azurecr.io/azureml/azureml_8a90a8ae4f21dcd7231b9244082417af
 ---> 2727e9c23274
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 96e266f68e07
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjUxMGI5NGJhLWU0NTMtNDQxNy05ODhiLWZiZGMzN2I1NWNhNyIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTEzODY2MCIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTEzODY2MCIsIndvcmtzcGFjZUlkIjoiYTk5ZmE0NzAtYWZjMy00M2Q4LWE0NzMtMWNjMTFlM2Q3MTQyIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in ca4a10f38629
 ---> 5861ac243918
Step 4/5 : RUN mv '/var/azureml-app/tmpv5af0tpg.py' /var/azureml-app/main.py


In [105]:
print('Local service port: {}'.format(local_service.port))

Local service port: 1234


In [46]:
print(local_service.get_logs())

2021-02-12T07:35:13,507446223+00:00 - rsyslog/run 
2021-02-12T07:35:13,508521724+00:00 - iot-server/run 
2021-02-12T07:35:13,511449026+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_c6bf3e53b0b53065cb9df10251065eb1/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c6bf3e53b0b53065cb9df10251065eb1/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c6bf3e53b0b53065cb9df10251065eb1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c6bf3e53b0b53065cb9df10251065eb1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c6bf3e53b0b53065cb9df10251065eb1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-02-12T07:35:13,516940730+00:00 - gunicorn/run 
EdgeHubC

In [106]:
import json

sample_input = json.dumps(
    {"data":
            [
                {
                    'age': 75.0,
                    'anaemia': 0,
                    'creatinine_phosphokinase': 582,
                    'diabetes': 0,
                    'ejection_fraction': 20,
                    'high_blood_pressure': 1,
                    'platelets': 265000.0,
                    'serum_creatinine': 1.9,
                    'serum_sodium': 130,
                    'sex': 1,
                    'smoking': 0,
                    'time': 4
                },
                {
                    'age': 55.0,
                    'anaemia': 0,
                    'creatinine_phosphokinase': 7861,
                    'diabetes': 0,
                    'ejection_fraction': 38,
                    'high_blood_pressure': 0,
                    'platelets': 263358.03,
                    'serum_creatinine': 1.1,
                    'serum_sodium': 136,
                    'sex': 1,
                    'smoking': 0,
                    'time': 6
                }
            ] 
        }
)

local_service.run(sample_input)

[0, 0]